In [1]:
import pandas as pd
import numpy as np

import os
import sys
module_path = os.path.abspath(os.path.join('..', '..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import warnings
warnings.filterwarnings('ignore')
import facts
from facts.clean import clean_dataset
from facts import valid_ifthens_with_coverage_correctness, rules2rulesbyif
from facts.models import customLogisticRegression
from facts.parameters import ParameterProxy
from facts.formatting import recourse_report_reverse, print_recourse_report, print_recourse_report_cumulative, print_recourse_report_KStest_cumulative
from facts.utils import load_rules_by_if
from facts.fairness_metrics_aggr import get_diff_table, get_comb_df, get_analysis_dfs,get_other_ranks_divided,get_map_metric_to_max_rank,get_diff_real_diff_drop

In [2]:
from pathlib import Path

computed_rules_file = Path("..") / ".." / "Pre-computed Results" / "IBM_AdCampaign" / "rulesAdCampaign_atomic.data"
computed_rules_with_cumulative_file = Path("..") / ".." / "Pre-computed Results" / "IBM_AdCampaign" / "rulesAdCampaign_cumulative.data"

In [3]:
cost_budget = 10
top_count = 20
c_inf = 5

# Dataset

In [4]:
DATAFILE = '../../../ad_campaign_data.csv' # location of dataset
random_state = None # change to something for exactly reproducible results

# as selected in "tutorial_bias_advertising" notebook
selected_features = ['religion','politics','gender', 'parents', 'ages', 'income', 'area', 'college_educated', 'homeowner']
cate_features = ['religion','politics','gender', 'parents', 'ages', 'income', 'area', 'college_educated']
num_features = ['homeowner']

# if not specified, set to 1
feature_weights = {"gender": 1, "parents": 1, "age": 1}

target_name = "true_conversion"
positive_label = 1
negative_label = 0

model_train_fraction = 0.7
learning_class_weights = {0: 0.5, 1: 265}

sensitive_attribute = "gender"
sensitive_alias = {"M": "Male", "F": "Female", "Unknown": "Unknown"}

ad_conversion_dataset = pd.read_csv(DATAFILE)
ad_conversion_dataset.gender = ad_conversion_dataset.gender.map(sensitive_alias)
ad_conversion_dataset.rename(columns = {'age':'ages'}, inplace = True)
ad_conversion_dataset = ad_conversion_dataset[selected_features + [target_name]]
ad_conversion_dataset.head()


,religion,politics,gender,parents,ages,income,area,college_educated,homeowner,true_conversion
0,Unknown,Unknown,Unknown,1,55-64,Unknown,Unknown,1,1,0
1,Other,Unknown,Unknown,1,55-64,Unknown,Urban,1,1,0
2,Unknown,Unknown,Female,1,55-64,Unknown,Unknown,1,1,0
3,Unknown,Unknown,Female,1,55-64,Unknown,Unknown,1,1,0
4,Unknown,Unknown,Female,1,55-64,Unknown,Urban,1,1,0


In [5]:
y = ad_conversion_dataset['true_conversion']
X = ad_conversion_dataset.drop('true_conversion', axis=1)

num_features = X._get_numeric_data().columns.to_list()
cate_features = X.select_dtypes(include=['object','category']).columns.to_list()
ord_features = []

# Rules

In [6]:
rules_with_atomic_correctness = load_rules_by_if(computed_rules_file)
rules_with_cumulative_correctness = load_rules_by_if(computed_rules_with_cumulative_file)

In [7]:
num_features = []
cate_features = ['religion','politics', 'parents', 'ages', 'income', 'area', 'college_educated', 'homeowner']
ord_features = []

feature_weights = {'religion':5,'politics':2, 'parents':3, 'ages':10, 'income':3, 'area':2, 'college_educated':3, 'homeowner':1}
features_with_binary_cost = cate_features
features_with_proportional_cost = num_features


comparators = facts.feature_change_builder(
    X,
    num_cols=features_with_proportional_cost,
    cate_cols=features_with_binary_cost,
    ord_cols=ord_features,
    feature_weights=feature_weights,
    num_normalization=False,
)
params = ParameterProxy(featureChanges=comparators)

# Macro Viewpoint

## Equal Cost of Effectiveness (Macro)

In [8]:
top_rules, subgroup_costs = facts.select_rules_subset(
    rules_with_atomic_correctness,
    metric="min-above-thr",
    sort_strategy="generic-sorting-ignore-exists-subgroup-empty",
    top_count=top_count,
    cor_threshold=0.7,
    filter_sequence = [
        "remove-contained",
        "remove-below-thr",        
        #"remove-fair-rules",
        "keep-only-min-change"
    ],
    params=params
)

print_recourse_report(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True,
    metric_name = 'Equal Cost of Effectiveness (Macro) (threshold = 0.7)'
)

If homeowner = 1:
	Protected Subgroup 'Male', 98.67% covered
		Make homeowner = 0 with effectiveness 94.80%.
		Aggregate cost of the above recourses = 1.00
	Protected Subgroup 'Female', 98.31% covered
		Make homeowner = 0 with effectiveness 93.00%.
		Aggregate cost of the above recourses = 1.00
	No bias!
If homeowner = 1, income = Unknown:
	Protected Subgroup 'Male', 95.78% covered
		Make homeowner = 0 with effectiveness 95.46%.
		Aggregate cost of the above recourses = 1.00
	Protected Subgroup 'Female', 95.46% covered
		Make homeowner = 0 with effectiveness 93.93%.
		Aggregate cost of the above recourses = 1.00
	No bias!
If homeowner = 1, parents = 1:
	Protected Subgroup 'Male', 92.94% covered
		Make homeowner = 0 with effectiveness 97.45%.
		Aggregate cost of the above recourses = 1.00
	Protected Subgroup 'Female', 93.06% covered
		Make homeowner = 0 with effectiveness 96.53%.
		Aggregate cost of the above recourses = 1.00
	No bias!
If college_educated = 1, homeowner = 1:
	Protected 

## Equal Choice for Recourse(Macro)

In [9]:
top_rules, subgroup_costs = facts.select_rules_subset(
    rules_with_atomic_correctness,
    metric="num-above-thr",
    sort_strategy="generic-sorting-ignore-forall-subgroups-empty",
    top_count=top_count,
    cor_threshold=0.7,
    filter_sequence = [
        "remove-contained",
        "remove-below-thr",
        "remove-fair-rules"
    ],
    params=params
)

print_recourse_report(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True,
    metric_name = 'Equal Choice for Recourse(Macro) (threshold = 0.7)'
)

If area = Unknown, religion = Unknown:
	Protected Subgroup 'Male', 53.09% covered
		Make area = Urban, religion = Christianity with effectiveness 85.47%.
		Make area = Urban, religion = Other with effectiveness 85.47%.
		Make area = Rural, religion = Christianity with effectiveness 90.52%.
		Make area = Rural, religion = Other with effectiveness 90.52%.
		Aggregate cost of the above recourses = -4.00
	Protected Subgroup 'Female', 48.86% covered
		No recourses for this subgroup!
		Aggregate cost of the above recourses = 0.00
	Bias against Female due to Equal Choice for Recourse(Macro) (threshold = 0.7). Unfairness score = 4.
If area = Unknown, homeowner = 1, religion = Unknown:
	Protected Subgroup 'Male', 51.81% covered
		Make homeowner = 0 with effectiveness 90.59%.
		Make homeowner = 0, religion = Christianity with effectiveness 99.24%.
		Make homeowner = 0, religion = Other with effectiveness 99.27%.
		Make area = Urban, religion = Christianity with effectiveness 85.15%.
		Make area 

# Micro Viewpoint

##  Equal Conditional Mean Recourse

In [10]:
top_rules, subgroup_costs = facts.select_rules_subset_cumulative(
    rules_with_cumulative_correctness,
    metric="fairness-of-mean-recourse-conditional",
    sort_strategy="generic-sorting-ignore-exists-subgroup-empty",
    top_count=top_count,
    filter_sequence = [
        "remove-contained",
        "remove-fair-rules"
    ],
    params=params
)

print_recourse_report_cumulative(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True,
    show_then_costs=True,
    metric_name = 'Equal Conditional Mean Recourse'
)

If ages = 55-64, area = Unknown, college_educated = 1, income = Unknown, parents = 0, politics = Unknown, religion = Unknown:
	Protected Subgroup 'Male', 2.77% covered
		Make parents = 1 with effectiveness 14.23% and counterfactual cost = 3.0.
		Make income = <100K, parents = 1 with effectiveness 14.23% and counterfactual cost = 6.0.
		Make college_educated = 0, parents = 1 with effectiveness 14.23% and counterfactual cost = 6.0.
		Make parents = 1, religion = Christianity with effectiveness 14.23% and counterfactual cost = 8.0.
		Make parents = 1, religion = Other with effectiveness 14.23% and counterfactual cost = 8.0.
		Make area = Urban, parents = 1, religion = Christianity with effectiveness 100.00% and counterfactual cost = 10.0.
		Make area = Urban, parents = 1, religion = Other with effectiveness 100.00% and counterfactual cost = 10.0.
		Make area = Rural, parents = 1, religion = Other with effectiveness 100.00% and counterfactual cost = 10.0.
		Make area = Rural, parents = 1, 

## Equal Cost of Effectiveness (Micro)

In [11]:
top_rules, subgroup_costs = facts.select_rules_subset_cumulative(
    rules_with_cumulative_correctness,
    metric="min-above-corr",
    sort_strategy="generic-sorting-ignore-exists-subgroup-empty",
    top_count=top_count,
    cor_threshold = 0.7,
    filter_sequence = [
        "remove-contained",
        #"remove-fair-rules",
        "keep-cheap-rules-above-thr-cor",
        
    ],
    params=params
)

print_recourse_report_cumulative(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True,
    show_then_costs=True,
    metric_name = 'Equal Cost of Effectiveness (Micro) (threshold = 0.7)'
)

If homeowner = 1:
	Protected Subgroup 'Male', 98.67% covered
		Make homeowner = 0 with effectiveness 94.80% and counterfactual cost = 1.0.
		Aggregate cost of the above recourses = 1.00
	Protected Subgroup 'Female', 98.31% covered
		Make homeowner = 0 with effectiveness 93.00% and counterfactual cost = 1.0.
		Aggregate cost of the above recourses = 1.00
	No bias!
If homeowner = 1, income = Unknown:
	Protected Subgroup 'Male', 95.78% covered
		Make homeowner = 0 with effectiveness 95.46% and counterfactual cost = 1.0.
		Aggregate cost of the above recourses = 1.00
	Protected Subgroup 'Female', 95.46% covered
		Make homeowner = 0 with effectiveness 93.93% and counterfactual cost = 1.0.
		Aggregate cost of the above recourses = 1.00
	No bias!
If homeowner = 1, parents = 1:
	Protected Subgroup 'Male', 92.94% covered
		Make homeowner = 0 with effectiveness 97.45% and counterfactual cost = 1.0.
		Aggregate cost of the above recourses = 1.00
	Protected Subgroup 'Female', 93.06% covered
		Make

## Equal Effectiveness

In [12]:
top_rules, subgroup_costs = facts.select_rules_subset_cumulative(
    rules_with_cumulative_correctness,
    metric="total-correctness",
    sort_strategy="generic-sorting-ignore-forall-subgroups-empty",
    top_count=top_count,
    filter_sequence = [
        "remove-contained",
        "remove-fair-rules",
    ],
    params=params
)

print_recourse_report_cumulative(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True,
    show_then_costs=True,
    correctness_metric=True,
    metric_name = 'Equal Effectiveness'
)

If ages = 55-64, area = Unknown, income = Unknown, parents = 1, politics = Unknown, religion = Unknown:
	Protected Subgroup 'Male', 43.91% covered
		Make income = <100K with effectiveness 0.00% and counterfactual cost = 3.0.
		Make religion = Christianity with effectiveness 0.00% and counterfactual cost = 5.0.
		Make religion = Other with effectiveness 0.00% and counterfactual cost = 5.0.
		Make area = Urban, religion = Christianity with effectiveness 94.30% and counterfactual cost = 7.0.
		Make area = Urban, religion = Other with effectiveness 94.30% and counterfactual cost = 7.0.
		Make area = Rural, religion = Other with effectiveness 100.00% and counterfactual cost = 7.0.
		Make area = Rural, religion = Christianity with effectiveness 100.00% and counterfactual cost = 7.0.
		Make income = <100K, religion = Christianity with effectiveness 100.00% and counterfactual cost = 8.0.
		Make income = <100K, religion = Other with effectiveness 100.00% and counterfactual cost = 8.0.
		Aggrega

## Fair Effectiveness-Cost Trade-Off

In [13]:
# taken from the other notebooks
affected_pop_sizes = {"Male": 94138, "Female": 180304}

top_rules,unfairness = facts.select_rules_subset_KStest(
    rules_with_cumulative_correctness,
    affected_pop_sizes,
    top_count=top_count
)

print_recourse_report_KStest_cumulative(
    top_rules,
    population_sizes=affected_pop_sizes,
    unfairness = unfairness,
    show_then_costs=True,
    metric_name = 'Fair Effectiveness-Cost Trade-Off'
    # show_cumulative_plots=True
)

If ages = 55-64, area = Unknown, income = Unknown, parents = 1, politics = Unknown, religion = Unknown:
	Protected Subgroup 'Male', 43.91% covered out of 94138
		Make income = <100K with effectiveness 0.00% and counterfactual cost = 3.0.
		Make religion = Christianity with effectiveness 0.00% and counterfactual cost = 5.0.
		Make religion = Other with effectiveness 0.00% and counterfactual cost = 5.0.
		Make area = Urban, religion = Christianity with effectiveness 94.30% and counterfactual cost = 7.0.
		Make area = Urban, religion = Other with effectiveness 94.30% and counterfactual cost = 7.0.
		Make area = Rural, religion = Other with effectiveness 100.00% and counterfactual cost = 7.0.
		Make area = Rural, religion = Christianity with effectiveness 100.00% and counterfactual cost = 7.0.
		Make income = <100K, religion = Christianity with effectiveness 100.00% and counterfactual cost = 8.0.
		Make income = <100K, religion = Other with effectiveness 100.00% and counterfactual cost = 8

## Equal Effectiveness within Budget (Micro)

In [14]:
top_rules, subgroup_costs = facts.select_rules_subset_cumulative(
    rules_with_cumulative_correctness,
    metric="max-upto-cost",
    sort_strategy="generic-sorting-ignore-exists-subgroup-empty",
    top_count=top_count,
    cost_threshold = cost_budget,
    filter_sequence = [
        "remove-contained",
        "remove-fair-rules",
        "remove-above-thr-cost"
    ],
    params=params
)

print_recourse_report_cumulative(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True,
    show_then_costs=True,
    correctness_metric = True,
    metric_name = 'Equal Effectiveness within Budget (Micro)'
)

If ages = 55-64, area = Unknown, income = Unknown, parents = 1, politics = Unknown, religion = Unknown:
	Protected Subgroup 'Male', 43.91% covered
		Make income = <100K with effectiveness 0.00% and counterfactual cost = 3.0.
		Make religion = Christianity with effectiveness 0.00% and counterfactual cost = 5.0.
		Make religion = Other with effectiveness 0.00% and counterfactual cost = 5.0.
		Make area = Urban, religion = Christianity with effectiveness 94.30% and counterfactual cost = 7.0.
		Make area = Urban, religion = Other with effectiveness 94.30% and counterfactual cost = 7.0.
		Make area = Rural, religion = Other with effectiveness 100.00% and counterfactual cost = 7.0.
		Make area = Rural, religion = Christianity with effectiveness 100.00% and counterfactual cost = 7.0.
		Make income = <100K, religion = Christianity with effectiveness 100.00% and counterfactual cost = 8.0.
		Make income = <100K, religion = Other with effectiveness 100.00% and counterfactual cost = 8.0.
		Aggrega

## Get ranking of subgroups based on metrics

In [15]:
from facts.fairness_metrics_aggr import make_table, auto_budget_calculation

In [16]:
rules_with_both = {}
for ifc, all_thens in rules_with_cumulative_correctness.items():
    new_all_thens = {}
    for sg, (cov, thens_cum) in all_thens.items():
        thens_atom = rules_with_atomic_correctness[ifc][sg][1]
        thens_atom_dict = {then: atom_cor for then, atom_cor in thens_atom}
        new_all_thens[sg] = (cov, [(then, thens_atom_dict[then], cumcor) for then, cumcor, _cost in thens_cum])
    rules_with_both[ifc] = new_all_thens

In [17]:
budgets = auto_budget_calculation(rules_with_cumulative_correctness, cor_thres=0.5, percentiles=[0.3, 0.6, 0.9])

df = make_table(
    rules_with_both,
    sensitive_attribute_vals=["Male", "Female"],
    effectiveness_thresholds=[0.3, 0.7],
    cost_budgets=budgets,
    params=params
)
# dropping 'weighted-average' and 'mean-cost-cinf'
df.drop(['weighted-average', 'mean-cost-cinf'], axis=1, inplace=True)

In [18]:
#table of differences between Male and Female for each metric
diff = get_diff_table(df, sensitive_attribute_vals=["Male", "Female"], with_abs = True)
diff = diff.set_index('subgroup')
ranked = diff.drop(columns=[('Fair Effectiveness-Cost Trade-Off','bias')]).mask(diff == 0).rank(ascending = False,axis=0,method='dense')
ranked = ranked.replace(np.nan,"Fair")

In [19]:
rev_bias_metrics = ['Equal Effectiveness', 'Equal Effectiveness within Budget']
comb_df = get_comb_df(df, ranked, diff, rev_bias_metrics, sensitive_attribute_vals=["Male", "Female"])
diff_real_val, _, = get_diff_real_diff_drop(df, diff, sensitive_attribute_vals=["Male", "Female"])
analysis_df,rank_analysis_df = get_analysis_dfs(comb_df, diff_real_val, rev_bias_metrics, ranked, sensitive_attribute_vals=["Male", "Female"], percentage=0.1)

In [20]:
comb_df.head()

(Equal Cost of Effectiveness(Macro), 0.3)  \
                                                                          rank   
subgroup                                                                         
homeowner = 1                                                             Fair   
homeowner = 1, politics = Unknown                                         Fair   
income = Unknown                                                          Fair   
income = Unknown, politics = Unknown                                      Fair   
homeowner = 1, income = Unknown                                           Fair   

                                                         \
                                     score bias against   
subgroup                                                  
homeowner = 1                          0.0         Fair   
homeowner = 1, politics = Unknown      0.0         Fair   
income = Unknown                       0.0         Fair   
income = Unknown, politics = Unknown   0.0         Fair   
homeowner = 1, income = Unknown        0.0         Fair   

                                     (Equal Cost of Effectiveness(Macro), 0.7)  \
                                                                          rank   
subgroup                                                                         
homeowner = 1                                                             Fair   
homeowner = 1, politics = Unknown                                         Fair   
income = Unknown                                                          Fair   
income = Unknown, politics = Unknown                                      Fair   
homeowner = 1, income = Unknown                                           Fair   

                                                         \
                                     score bias against   
subgroup                                                  
homeowner = 1                          0.0         Fair   
homeowner = 1, politics = Unknown      0.0         Fair   
income = Unknown                       0.0         Fair   
income = Unknown, politics = Unknown   0.0         Fair   
homeowner = 1, income = Unknown        0.0         Fair   

                                     (Equal Choice for Recourse, 0.3)        \
                                                                 rank score   
subgroup                                                                      
homeowner = 1                                                    Fair   0.0   
homeowner = 1, politics = Unknown                                Fair   0.0   
income = Unknown                                                 Fair   0.0   
income = Unknown, politics = Unknown                             Fair   0.0   
homeowner = 1, income = Unknown                                  Fair   0.0   

                                                   \
                                     bias against   
subgroup                                            
homeowner = 1                                Fair   
homeowner = 1, politics = Unknown            Fair   
income = Unknown                             Fair   
income = Unknown, politics = Unknown         Fair   
homeowner = 1, income = Unknown              Fair   

                                     (Equal Choice for Recourse, 0.7)  ...  \
                                                                 rank  ...   
subgroup                                                               ...   
homeowner = 1                                                    Fair  ...   
homeowner = 1, politics = Unknown                                Fair  ...   
income = Unknown                                                 Fair  ...   
income = Unknown, politics = Unknown                             Fair  ...   
homeowner = 1, income = Unknown                                  Fair  ...   

                                     (Equal Cost of Effectiveness(Micro), 0.3)  \
            

In [21]:
analysis_df

,Rank = 1 Count,Male bias against Count,Female bias against Count,Top 10% Male bias against Count,Top 10% Female bias against Count
"(Equal Cost of Effectiveness(Macro), 0.3)",427,16,429,0,44
"(Equal Cost of Effectiveness(Macro), 0.7)",264,0,264,0,26
"(Equal Choice for Recourse, 0.3)",2,16,645,0,66
"(Equal Choice for Recourse, 0.7)",384,0,392,0,39
Equal Effectiveness,15,167,1072,0,123
"(Equal Effectiveness within Budget, 1.0)",1,20,402,0,42
"(Equal Effectiveness within Budget, 5.0)",10,151,989,0,114
"(Equal Cost of Effectiveness(Micro), 0.3)",427,16,429,0,44
"(Equal Cost of Effectiveness(Micro), 0.7)",264,0,264,0,26
Equal(Conditional) Mean Recourse,108,278,554,9,74


In [22]:
max_ranks = get_map_metric_to_max_rank(ranked)
rank_divided = get_other_ranks_divided(rank_analysis_df, max_ranks)
rank_divided

,"(Equal Cost of Effectiveness(Macro), 0.3)","(Equal Cost of Effectiveness(Macro), 0.7)","(Equal Choice for Recourse, 0.3)","(Equal Choice for Recourse, 0.7)",Equal Effectiveness,"(Equal Effectiveness within Budget, 1.0)","(Equal Effectiveness within Budget, 5.0)","(Equal Cost of Effectiveness(Micro), 0.3)","(Equal Cost of Effectiveness(Micro), 0.7)",Equal(Conditional) Mean Recourse,"(Fair Effectiveness-Cost Trade-Off, value)"
"(Equal Cost of Effectiveness(Macro), 0.3)",NaN,0.70,0.483,0.600,0.167,1.000,0.276,0.250,0.70,0.487,0.154
"(Equal Cost of Effectiveness(Macro), 0.7)",0.250,NaN,0.350,0.333,0.082,1.000,0.210,0.250,0.50,0.506,0.079
"(Equal Choice for Recourse, 0.3)",0.250,1.00,NaN,1.000,0.730,1.000,1.000,0.250,1.00,0.037,0.338
"(Equal Choice for Recourse, 0.7)",0.500,0.65,0.333,NaN,0.296,0.851,0.385,0.500,0.65,0.566,0.273
Equal Effectiveness,0.250,0.50,0.333,0.333,NaN,1.000,0.205,0.250,0.50,0.002,0.001
"(Equal Effectiveness within Budget, 1.0)",1.000,1.00,1.000,1.000,0.714,NaN,0.671,1.000,1.00,0.305,0.395
"(Equal Effectiveness within Budget, 5.0)",0.250,0.50,0.333,0.333,0.001,1.000,NaN,0.250,0.50,0.002,0.001
"(Equal Cost of Effectiveness(Micro), 0.3)",0.250,0.70,0.483,0.600,0.167,1.000,0.276,NaN,0.70,0.487,0.154
"(Equal Cost of Effectiveness(Micro), 0.7)",0.250,0.50,0.350,0.333,0.082,1.000,0.210,0.250,NaN,0.506,0.079
Equal(Conditional) Mean Recourse,0.525,0.75,0.650,0.700,0.250,1.000,0.408,0.525,0.75,NaN,0.267
